In [ ]:
%pip install requests
%pip install attrs
%pip install python-dotenv
%pip install shapely
%pip install ipyleaflet
%pip install limiter

In [8]:
import configargparse

env = dict()
with open("urbiotica.ini", "r", encoding="utf-8") as infile:
    for line in infile.readlines():
        line = line.strip()
        if line:
            key, val = [x.strip() for x in line.split("=")]
            key = key.replace("-", "_").upper()
            env[key] = val

env.keys()

dict_keys(['API_ORGANISM', 'API_USERNAME', 'API_PASSWORD', 'ORION_USERNAME', 'ORION_PASSWORD', 'ORION_SERVICE', 'ORION_SUBSERVICE', 'API_URL', 'KEYSTONE_URL', 'ORION_URL'])

In [9]:
from datetime import datetime, timedelta, timezone
from operator import itemgetter, attrgetter
from typing import Dict, Mapping, Any, List, Generator, Iterator
import math

import attr
from limiter import Limiter, get_limiter, limit_rate
from orion import Session, ContextBroker

JsonDict = Dict[str, Any]
JsonList = List[JsonDict]

@attr.s(auto_attribs=True)
class Api(object):

    endpoint: str
    organism: str
    token: str
    bucket: Limiter

    @classmethod
    def login(cls, session: Session, endpoint: str, organism: str, username: str, password: str):
        # API is rate limited to 100 requests per minute
        bucket = get_limiter(rate = 100.0/60.0, capacity=100)
        with limit_rate(bucket):
            auth = session.get(f'{endpoint}/v2/auth/{organism}/{username}/{password}')
        if auth.status_code != 200:
            raise ValueError('Invalid username of password')
        return cls(endpoint, organism, auth.text.strip('"'), bucket)

    def projects(self, session: Session) -> Dict[str, 'Project']:
        url = f'{self.endpoint}/v2/organisms/{self.organism}/projects'
        with limit_rate(self.bucket):
            prj = session.get(url, headers={ 'IDENTITY_KEY': self.token })
        if prj.status_code != 200:
            raise ValueError(f'Failed to retrieve projects for organism {self.organism}')
        return { item['projectid']: Project.new(self, item) for item in prj.json() }

    def _query(self, session: Session, projectid: str, path: str, attrib: str, debug=False) -> JsonDict:
        url = f'{self.endpoint}/v2/organisms/{self.organism}/projects/{projectid}/{path}'
        with limit_rate(self.bucket):
            its = session.get(url, headers={ 'IDENTITY_KEY': self.token })
        if its.status_code != 200:
            raise ValueError(f'Failed to retrieve {path} for project {projectid}')
        if debug:
            print("JSON: ", its.json())
        return { item[attrib]: item for item in its.json() }


@attr.s(auto_attribs=True)
class Project(object):

    api: Api
    projectid: str
    name: str
    description: str
    timezone: str

    @classmethod
    def new(cls, api: Api, project: JsonDict):
        return cls(api, project['projectid'], project['name'], project['description'], project['timezone'])

    def parkings(self, session: Session) -> JsonDict:
        return self.api._query(session, self.projectid, 'parkings', 'pomid')
    
    def zones(self, session: Session) -> JsonDict:
        return self.api._query(session, self.projectid, 'zones', 'zoneid')
    
    def spots(self, session: Session) -> JsonDict:
        return self.api._query(session, self.projectid, 'spots', 'pomid')

    def devices(self, session: Session, zoneid: str) -> JsonDict:
        return self.api._query(session, self.projectid, f'zones/{zoneid}/devices', 'elementid')

    def rotations(self, session: Session, pomid: str, from_dt: datetime, to_dt: datetime) -> JsonList:
        fromiso = datetime.isoformat(from_dt.replace(microsecond=0))
        toiso = datetime.isoformat(to_dt.replace(microsecond=0))
        result = dict()
        poms = self.api._query(session, self.projectid, f'spots/{pomid}/rotations/finished/{fromiso}/{toiso}', 'pomid')
        rotations = list(itertools.chain(*(({
            'pomid': pom['pomid'],
            'start': datetime.fromisoformat(item['start']),
            'end': datetime.fromisoformat(item['end'])
        } for item in pom['rotations']) for pom in poms.values())))
        return Project._sortby(rotations, 'start')

    def vehicles(self, session: Session, pomid: str, from_dt: datetime, to_dt: datetime) -> JsonList:
        fromts = math.floor(from_dt.timestamp())
        tots = math.ceil(to_dt.timestamp())+1
        result = dict()
        poms = self.api._query(session, self.projectid, f'spots/{pomid}/phenomenons/vehicle_ctrl?start={fromts}&end={tots}', 'pomid', True)
        #poms = self.api._query(self.projectid, f'spots/{pomid}/phenomenons/vehicle_ctrl', 'pomid', True)
        measurements = list(itertools.chain(*(({
            'pomid': pom['pomid'],
            'lstamp': datetime.fromtimestamp(int(item['lstamp']) // 1000, tz=timezone.utc),
            'value': item['value']
        } for item in pom['measurements']) for pom in poms.values())))
        return Project._sortby(measurements, 'lstamp')

    @staticmethod
    def _sortby(items: JsonList, field: str) -> JsonList:
        items.sort(key=itemgetter(field))
        return items


In [11]:
session = Session()
api = Api.login(session, env['API_URL'], env['API_ORGANISM'], env['API_USERNAME'], env['API_PASSWORD'])
api.token

'HtjctFXRrYEquJccsTuVVWeq0UfSRwNnapTDIfoLJAzh3XvX5fBeDvuGOvm5eSbB'

In [5]:
from collections import defaultdict
import itertools

@attr.s(auto_attribs=True)
class Timestamp(object):

    date: datetime
    events: List[str]

    @classmethod
    def new(cls, date: datetime, events: List[str]):
        return cls(date, events)
    
    def __str__(self) -> str:
        return "\n".join(itertools.chain(
            (self.date.isoformat(),),
            (f'  {event}' for event in self.events)
        ))

@attr.s(auto_attribs=True)
class Timeline(object):

    events: JsonList

    @classmethod
    def new(cls, rotations: JsonList, vehicles: JsonList):
        events: Dictionary[datetime, List[str]] = defaultdict(list)
        estados = {
            '0': 'ocupada',
            '1': 'libre',
            '-1': 'en estado desconocido',
        }
        for rotation in rotations:
            events[rotation['start']].append("Comienza rotacion")
            events[rotation['end']].append("Termina rotacion de las %s" % rotation['start'])
        for vehicle in vehicles:
            value = vehicle['value']
            events[vehicle['lstamp']].append('Plaza %s (%s)' % (estados[value], value))
        stamps = [ Timestamp.new(date, info) for date, info in events.items() ]
        stamps.sort(key=attrgetter('date'))
        return cls(stamps)
    
    def __str__(self) -> str:
        return "\n".join(str(event) for event in self.events)


In [ ]:
for prjid, prj in api.projects(session).items():
    for pomid, superpom in prj.parkings(session).items():
        for pom in superpom['poms']:
            now = datetime.now()
            bef = now - timedelta(days=1)
            rots = prj.rotations(session, pom['pomid'], bef, now)
            vehi = prj.vehicles(session, pom['pomid'], bef, now)
            print(vehi)
            ts = Timeline.new(rots, vehi)
            print(str(ts))
            break
        break
    break


In [12]:
projects = api.projects(session)

In [13]:
project0 = projects[tuple(projects.keys())[0]]

In [14]:
import itertools

parkings = project0.parkings(session)
subpoms = [[pom['pomid'] for pom in parking['poms']] for parking in parkings.values()]
parking_spots = frozenset(itertools.chain(*subpoms))
parking_by_pom = dict()
for parking in parkings.values():
    for pom in parking['poms']:
        parking_by_pom[pom['pomid']] = parking
print("PARKINGS - SPOTS: ", parking_spots)

PARKINGS - SPOTS:  frozenset({45866, 45867, 45868, 45869, 45870, 45871, 45872, 45873, 45874, 45875, 45876, 45877, 45878, 45879, 45880, 45881, 45882, 45883, 45884, 45885, 45886, 45887, 45888, 45889, 45890, 45891})


In [15]:
zones = project0.zones(session)
print("ZONES:", zones)


ZONES: {733: {'organismid': 'org387236', 'projectid': 'prj4dd9a3', 'zoneid': 733, 'description': 'CyD', 'lat_ne': 38.7057482126756, 'long_ne': -0.475437696205967, 'lat_sw': 38.6986483951214, 'long_sw': -0.482798040859641, 'epsg': '4326', 'image': 'prj4dd9a3_733.jpg', 'main_zone': False}}


In [16]:
zone0 = zones[tuple(zones.keys())[0]]
zone0

{'organismid': 'org387236',
 'projectid': 'prj4dd9a3',
 'zoneid': 733,
 'description': 'CyD',
 'lat_ne': 38.7057482126756,
 'long_ne': -0.475437696205967,
 'lat_sw': 38.6986483951214,
 'long_sw': -0.482798040859641,
 'epsg': '4326',
 'image': 'prj4dd9a3_733.jpg',
 'main_zone': False}

In [17]:
devices = project0.devices(session, zone0['zoneid'])
device_spots = frozenset(item['pomid'] for item in devices.values())
print("DEVICES_POMS: ", device_spots)

DEVICES_POMS:  frozenset({45866, 45867, 45868, 45869, 45870, 45871, 45872, 45873, 45874, 45875, 45876, 45877, 45878, 45879, 45880, 45881, 45882, 45883, 45884, 45885, 45886, 45887, 45888, 45889, 45890, 45891, 45892, 45893, 45894, 45895, 45896, 45897})


In [18]:
spots = project0.spots(session)
spot_ids = frozenset(item['pomid'] for item in spots.values())
print("SPOTS:", spot_ids)

SPOTS: frozenset({45866, 45867, 45868, 45869, 45870, 45871, 45872, 45873, 45874, 45875, 45876, 45877, 45878, 45879, 45880, 45881, 45882, 45883, 45884, 45885, 45886, 45887, 45888, 45889, 45890, 45891})


In [19]:
spot0 = spots[tuple(spots.keys())[0]]
spot1 = spots[tuple(spots.keys())[1]]
print("SPOT 0: ", spot0)
print("SPOT 1: ", spot1)
device0 = devices[spot0['elementid']]
print("DEVICE 0", device0)
parking0 = parking_by_pom[spot0['pomid']]
print("PARKING 0: ", parking0)

SPOT 0:  {'pomid': 45888, 'name': 'S-23', 'type': 'uspot', 'latitude': '38.7039574713965', 'longitude': '-0.477023949799113', 'attributes': [{'attributeid': 'algorithm_status', 'value': '0', 'last_update': 1625472336556}], 'elementid': '0100001b2c0afad4', 'status_location': 2, 'entity': 'spots'}
SPOT 1:  {'pomid': 45889, 'name': 'S-24', 'type': 'uspot', 'latitude': '38.7040458009425', 'longitude': '-0.476922383447501', 'attributes': [{'attributeid': 'algorithm_status', 'value': '0', 'last_update': 1625470049078}], 'elementid': '0100001b2b245fbf', 'status_location': 2, 'entity': 'spots'}
DEVICE 0 {'organismid': 'org387236', 'projectid': 'prj4dd9a3', 'zoneid': 733, 'pomid': 45888, 'name': 'S-23', 'elementid': '0100001b2c0afad4', 'status': {'status': 'up', 'lastseen': 1629364248460}, 'type': 'uspot', 'attributes': [{'attributeid': 'algorithm_status', 'value': '0', 'last_update': 1625472336556}]}
PARKING 0:  {'pomid': 45898, 'name': 'CyD', 'type': 'uspot', 'latitude': '38.7020857751693', '

In [20]:
from shapely.geometry import Polygon

points = [[ float(pom['latitude']), float(pom['longitude'])] for pom in parking0['poms']]
print("POINTS: ", points)
hull = Polygon(points).buffer(0.0001).minimum_rotated_rectangle

POINTS:  [[38.7039574713965, -0.477023949799113], [38.7040458009425, -0.476922383447501], [38.7045028269765, -0.476394933608962], [38.7045483233505, -0.476342616685345], [38.699623226988, -0.482021575538852], [38.6996655785243, -0.481972881817549], [38.6997079390165, -0.481924186573697], [38.6997590231559, -0.481865552517785], [38.6997976996308, -0.481821088745381], [38.6998365783307, -0.481776393625468], [38.6998744879126, -0.481732718955235], [38.7005222818808, -0.480983981083665], [38.7005557444634, -0.480945517194455], [38.7005891980041, -0.480907052436327], [38.7012650937665, -0.480126811247066], [38.701298547033, -0.480088346872198], [38.7013320002607, -0.480049881312377], [38.7025647427481, -0.478628367975212], [38.7025981954426, -0.478589901049265], [38.7026316571509, -0.478551434919845], [38.7029752900474, -0.478155391321419], [38.7030087425939, -0.478116925101623], [38.7030422041047, -0.478078457380424], [38.7037705694468, -0.477238845768122], [38.7038040306369, -0.4772003772

In [82]:
from ipyleaflet import Map, Marker, Polygon

center = (float(parking0['latitude']), float(parking0['longitude']))

m = Map(center=center, zoom=16)

for point in points:
    m.add_layer(Marker(location=point, draggable=True))


area = Polygon(
    locations=list(hull.exterior.coords),
    color="green",
    fill_color="green"
)
m.add_layer(area)

display(m)

Map(center=[38.7020857751693, -0.479182096112105], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [21]:
keystoneURL = env["KEYSTONE_URL"]
orionURL = env["ORION_URL"]
service = env["ORION_SERVICE"]
subservice = env["ORION_SUBSERVICE"]
username = env["ORION_USERNAME"]
password = env["ORION_PASSWORD"]

import logging
import sys 

root = logging.getLogger()
root.setLevel(logging.DEBUG)
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
root.addHandler(handler)

session = Session()
logging.info("Authenticating to url %s, service %s, username %s", keystoneURL, service, username)
cb = ContextBroker(
    keystoneURL=keystoneURL,
    orionURL=orionURL,
    service=service,
    subservice=subservice)
cb.auth(session, username, password)


Authenticating to url https://auth.iotplatform.telefonica.com:15001, service sc_alcoi_int, username alcoi_int_admin
Starting new HTTPS connection (1): auth.iotplatform.telefonica.com:15001
https://auth.iotplatform.telefonica.com:15001 "POST /v3/auth/tokens HTTP/1.1" 201 1712


In [12]:
cb.batch(session, [{'id': 'pomid:45891', 'type': 'ParkingSpot', 'occupied': {'type': 'Number', 'value':None}}])

Starting new HTTPS connection (1): cb.iotplatform.telefonica.com:10027
Starting new HTTPS connection (1): cb.iotplatform.telefonica.com:10027
https://cb.iotplatform.telefonica.com:10027 "POST /v2/op/update HTTP/1.1" 204 0
https://cb.iotplatform.telefonica.com:10027 "POST /v2/op/update HTTP/1.1" 204 0


In [7]:
@attr.s(auto_attribs=True)
class SpotIterator(object):

    # IDs of POM, device and zone
    pomid: int
    deviceid: str
    zoneid: str

    # Orion entity IDs
    entityid: str
    deviceentityid: str
    zoneentityid: str

    # Other attributes of spot
    name: str
    coords: List[float]

    # Time range and events in that range
    from_ts: datetime
    to_ts: datetime
    events: JsonList

    @classmethod
    def collect(cls, session: Session, project: Project, cb: ContextBroker, pom: JsonDict, device: JsonDict, to_ts: datetime):
        """Collect vehicle_ctrl events for the given pomid between most recent update, and to_ts"""
        pomid = pom['pomid']
        name = pom['name']
        logging.info("Collecting vehicle_ctrl events from pom %s (id %d)", name, pomid)
        deviceid = device['elementid']
        zoneid = device['zoneid']
        coords = [float(pom['latitude']), float(pom['longitude'])]
        entityid = f'pomid:{pomid}'
        deviceentityid = f'elementid:{deviceid}'
        zoneentityid = f'zoneid:{zoneid}'
        logging.info(f'Getting latest occupancyModified for entity {entityid}')
        entity = cb.get(session, entityID=entityid, entityType="ParkingSpot")
        from_ts = to_ts - timedelta(days=1)
        if entity is not None and 'occupancyModified' in entity:
            from_ts = datetime.fromisoformat(entity['occupancyModified']['value'].replace('Z','+00:00'))
        logging.info(f'Getting events for pomid {pomid} between {from_ts} and {to_ts}')
        events = project.vehicles(session, pomid, from_ts, to_ts)
        return cls(
            pomid=pomid,name=name,deviceid=deviceid,zoneid=zoneid,coords=coords,entityid=entityid,deviceentityid=deviceentityid,zoneentityid=zoneentityid,from_ts=from_ts,to_ts=to_ts,events=events
        )

    def __iter__(self) -> Generator[JsonDict, None, None]:
        """Iterate on vehicle_ctrl events generating ParkingSpot entity updates"""
        if self.events:
            for event in self.events:
                timeinstant = event['lstamp'].isoformat()
                occupied = int(event['value'])
                yield {
                    'id': self.entityid,
                    'type': 'ParkingSpot',
                    'TimeInstant': {
                        'type': 'DateTime',
                        'value': timeinstant,
                    },
                    'occupancyModified': {
                        'type': 'DateTime',
                        'value': timeinstant,
                    },
                    'name': {
                        'type': 'Text',
                        'value': self.name
                    },
                    'status': {
                        'type': 'Text',
                        'value': 'free' if occupied == 0 else ('occupied' if occupied == 1 else 'unknown'),
                    },
                    'refOnStreetParking': {
                        'type': 'Text',
                        'value': self.zoneentityid
                    },
                    'refDevice': {
                        'type': 'Text',
                        'value': self.deviceentityid
                    },
                    'location': {
                        'type': 'geo:json',
                        'value': {
                            'type': 'Point',
                            # HACK: Urbo coordinate system is "swapped"
                            'coordinates': [self.coords[1], self.coords[0]]
                        }
                    },
                    'occupied': {
                        'type': 'Number',
                        'value': occupied if occupied >= 0 else None
                    }
                }


In [22]:
from shapely.geometry import Polygon

def zone_to_entity(zone: JsonDict, zone_poms: JsonDict):
    """Collect zone information for the given zone"""
    zoneid = zone['zoneid']
    name = zone['description']
    location = [
        (float(zone['lat_ne'])  + float(zone['lat_sw']))/2,
        (float(zone['long_ne']) + float(zone['long_sw']))/2
    ]
    points = [[ float(pom['latitude']), float(pom['longitude'])] for pom in zone_poms]
    area = Polygon(points).buffer(0.0001).minimum_rotated_rectangle.exterior.coords
    return {
        'id': f'zoneid:{zoneid}',
        'type': 'OnStreetParking',
        'name': {
            'type': 'Text',
            'value': name
        },
        'location': {
            'type': 'geo:json',
            'value': {
                'type': 'Point',
                # HACK: Urbo swaps latitude and longitude...
                'coordinates': [location[1], location[0]]
            }
        },
        'polygon': {
            'type': 'geox:json',
            'value': {
                'type': 'Polygon',
                # HACK: Urbo swaps latitude and longitude...
                'coordinates': [[[item[1], item[0]] for item in area]]
            }
        },
        'totalSpotNumber': {
            'type': 'Number',
            'value': len(zone_poms)
        }
    }


In [23]:
from concurrent.futures import ThreadPoolExecutor
from collections import defaultdict

all_zones = dict()
poms_by_zone = defaultdict(list)
pom_params = list()
now_ts = datetime.now()
for project_id, project in api.projects(session).items():
    zones = project.zones(session)
    all_zones.update(zones)
    devices = dict()
    for zoneid, zone in zones.items():
        devices.update(project.devices(session, zoneid))
    for pomid, pom in project.spots(session).items():
        poms_by_zone[devices[pom['elementid']]['zoneid']].append(pom)
        pom_params.append({
            'session': session,
            'project': project,
            'cb': cb,
            'pom': pom,
            'device': devices[pom['elementid']],
            'to_ts': now_ts,
        })

Starting new HTTP connection (1): 172.18.176.1:3128
http://172.18.176.1:3128 "GET http://api.urbiotica.net/v2/organisms/org387236/projects HTTP/1.1" 200 193
http://172.18.176.1:3128 "GET http://api.urbiotica.net/v2/organisms/org387236/projects/prj4dd9a3/zones HTTP/1.1" 200 255
http://172.18.176.1:3128 "GET http://api.urbiotica.net/v2/organisms/org387236/projects/prj4dd9a3/zones/733/devices HTTP/1.1" 200 8452
http://172.18.176.1:3128 "GET http://api.urbiotica.net/v2/organisms/org387236/projects/prj4dd9a3/spots HTTP/1.1" 200 6915


In [ ]:
with ThreadPoolExecutor(max_workers=8) as pool:
    iterators = pool.map(lambda p: SpotIterator.collect(**p), pom_params)

In [24]:
def rotate(iterators: List[Iterator[JsonDict]]):
    """Rotate a set of iterators yielding one item from each"""
    iterables = [iter(item) for item in iterators]
    while len(iterables) > 0:
        depleted = False
        for index, item in enumerate(iterables):
            try:
                entity = next(item)
            except StopIteration:
                iterables[index] = None
                depleted = True
            else:
                yield entity
        if depleted:
            depleted = False
            iterables = [item for item in iterables if item is not None]
    

cb.batch(session, rotate(iterators))

NameError: name 'iterators' is not defined

In [25]:
entities = list()
for zoneid, zone in all_zones.items():
    zone_poms = poms_by_zone[zoneid]
    entities.append(zone_to_entity(zone, zone_poms))

cb.batch(session, entities)

Starting new HTTPS connection (1): cb.iotplatform.telefonica.com:10027
https://cb.iotplatform.telefonica.com:10027 "POST /v2/op/update HTTP/1.1" 204 0
